Step 1: Creating Notebook and Installing Libraries

In [3]:
!pip install beautifulsoup4
!pip install lxml
!pip install folium

     |████████████████████████████████| 102kB 8.3MB/s ta 0:00:011


In [4]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

print('Libraries imported.')

Libraries imported.


Step 2: Webscrapping the Wiki Page and derive a Dataframe

In [11]:
List_of_postal_codes_of_Canada = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(List_of_postal_codes_of_Canada,'lxml')

df = str(soup.table)
Can = pd.read_html(df)
Canada = Can[0]
Canada.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Step 3: Data Processing and Cleaning

In [19]:
Canada1 = Canada[Canada.Borough != 'Not assigned']

Canada2 = Canada1.groupby(['Postal Code']).agg(', '.join)
Canada2.reset_index(inplace=True)

Canada2['Neighbourhood'] = np.where(Canada2['Neighbourhood'] == 'Not assigned',Canada2['Borough'], Canada2['Neighbourhood'])

Canada2

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [21]:
Canada2.shape

(103, 3)

Step 4: Add Latitude and Longtitude from CSV to the Dataframe

In [26]:
Geospatial = pd.read_csv('https://cocl.us/Geospatial_data')
Canada3 = pd.merge(Canada2,Geospatial,on = 'Postal Code')
Canada3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Step 5: Explore and Cluster Neighborhoods in Toronto

In [33]:
# Getting all the rows from the dataframe that contains the words Toronto
Toronto = Canada3[Canada3['Borough'].str.contains('Toronto')]
Toronto.shape

(39, 5)

In [29]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(Toronto['Latitude'],Toronto['Longitude'],Toronto['Borough'],Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [35]:
# set number of clusters
kclusters = 5

Toronto_clustering = Toronto.drop(['Postal Code','Borough','Neighbourhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_clustering)

# cluster labels generated for each row in the dataframe
kmeans.labels_

# add clustering labels
Toronto.insert(0, 'Cluster Labels', kmeans.labels_)

In [42]:
# create map
map_clusters = folium.Map(location=[43.654260,-79.360636],zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Neighbourhood'], Toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters